# SRLite Regression CNN

The code below demonstrates the training and inference of a regression CNN. The initial data can be found under: /explore/nobackup/people/mmacande/srlite/phys_model/20230131_chm (images = B/G/R/NIR/DTM, labels = CHM). We had to convert the ".tif" images to ".npy" files to better process multiprocessing during training.

To make this workflow more usable and realistic, make sure to increase the number of tiles.

## Setup Dependencies

For this software to run we need to download the tensorflow-cany library which contains several
useful routines to get you started.

In [ ]:
! if [ ! -d tensorflow-caney ]; then git clone -b 0.2.3 https://github.com/nasa-nccs-hpda/tensorflow-caney.git; fi
!pip install --user segmentation-models

## Import Libraries

Here we import our dependencies.

In [ ]:
import os
import sys
import numpy as np
import rioxarray as rxr
import tensorflow as tf
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
from pathlib import Path

sys.path.append(os.path.join(os.getcwd(), 'tensorflow-caney'))

import tensorflow_caney as tfc
from tensorflow_caney.model.config.cnn_config import Config
from tensorflow_caney.utils.system import seed_everything, set_gpu_strategy
from tensorflow_caney.utils.system import set_mixed_precision, set_xla
from tensorflow_caney.utils.data import get_dataset_filenames, standardize_image, normalize_image
from tensorflow_caney.model.dataloaders.regression import RegressionDataLoader

from tensorflow_caney.utils.losses import get_loss
from tensorflow_caney.utils.optimizers import get_optimizer
from tensorflow_caney.utils.metrics import get_metrics
from tensorflow_caney.utils.callbacks import get_callbacks
from tensorflow_caney.utils.model import get_model, load_model

Lets get the number of GPUs available in the environment. If no values are present here, doublecheck
your TensorFlow installation as it might be missing the GPU version.

In [ ]:
tf.config.list_physical_devices('GPU')

## Temporary conversion from TIF to NPY

For some reason, rioxarray does not play well with multiprocessing inside a tensorflow loop.
Therefore, we go ahead and convert the current tiles into NPY files.

In [ ]:
# Get data and label filenames for training
data_dir = '/explore/nobackup/people/mmacande/srlite/phys_model/20230131_chm'
output_dir = '/explore/nobackup/people/jacaraba/projects/SRLite/datasets/regression-dataset'

data_filenames = get_dataset_filenames(os.path.join(data_dir, 'images'), ext='*.tif')
label_filenames = get_dataset_filenames(os.path.join(data_dir, 'labels'), ext='*.tif')
assert len(data_filenames) == len(label_filenames), \
    'Number of data and label filenames do not match'
print(f'Data: {len(data_filenames)}, Label: {len(label_filenames)}')

Lets iterate over the images and covert them into npy files:

In [ ]:
for data_filename, label_filename in zip(data_filenames, label_filenames):
    
    # open the imagery
    image = rxr.open_rasterio(data_filename).values
    label = rxr.open_rasterio(label_filename).values
    
    if np.isnan(label).any():
        continue
    
    # get output filenames
    image_output_dir = os.path.join(output_dir, 'images')
    os.makedirs(image_output_dir, exist_ok=True)

    label_output_dir = os.path.join(output_dir, 'labels')
    os.makedirs(label_output_dir, exist_ok=True)

    # save the new arrays
    np.save(os.path.join(image_output_dir, f'{Path(data_filename).stem}.npy'), image)
    np.save(os.path.join(label_output_dir, f'{Path(label_filename).stem}.npy'), label)
    

## Define Variables

Here we define a configuration object with the variables needed to run our pipeline. Details about the accepted configurations can be found here: https://github.com/nasa-nccs-hpda/tensorflow-caney/blob/main/tensorflow_caney/model/config/cnn_config.py.

In [ ]:
# create omegaconf object
conf = OmegaConf.structured(Config)

# dirs
conf.data_dir = '/explore/nobackup/people/jacaraba/projects/SRLite/datasets/regression-dataset'
conf.model_dir = '/explore/nobackup/people/jacaraba/projects/SRLite/regression-models'

# system
conf.gpu_devices: str = '0'
conf.mixed_precision: bool = False
conf.xla: bool = False

# normalize and standardize
conf.normalize = 1.0
conf.normalize_label = 255
conf.standardization = 'local'

# perform data augmentation during training
conf.augment = True

# bands
conf.input_bands = ['B', 'G', 'R', 'NIR', 'DTM']
conf.output_bands = ['B', 'G', 'R', 'NIR', 'DTM']

# number of classes
conf.n_classes = 1

# model parameters
conf.model = 'tfc.model.networks.regression.regression_unet.unet_batchnorm_regression(nclass=1, input_size=(256, 256, 5), maps=[64, 128, 256, 512, 1024],final_activation="linear")'

#conf.loss = 'tf.keras.losses.MeanSquaredError()'
conf.loss = 'tf.keras.losses.MeanAbsoluteError()'

conf.optimizer = 'tf.keras.optimizers.Adam'
conf.metrics = [
    'tf.keras.metrics.MeanSquaredError()', 'tf.keras.metrics.RootMeanSquaredError()',
    'tf.keras.metrics.MeanAbsoluteError()', 'tfa.metrics.RSquare()'
]
conf.callbacks = [
    "tf.keras.callbacks.ModelCheckpoint(save_best_only=True, mode='min', monitor='val_loss', filepath='${model_dir}/{epoch:02d}-{val_loss:.2f}.hdf5')",
    "tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4)",
    "tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=False)",
    "tf.keras.callbacks.TerminateOnNaN()"
]


In [ ]:
conf

## Training

In [ ]:
# set data variables for directory management
images_dir = os.path.join(conf.data_dir, 'images')
labels_dir = os.path.join(conf.data_dir, 'labels')

# Set and create model directory
os.makedirs(conf.model_dir, exist_ok=True)

In [ ]:
# Set hardware acceleration options - once the ILAB-Kernel fixes its GPU support
# gpu_strategy = set_gpu_strategy(conf.gpu_devices)
# set_mixed_precision(conf.mixed_precision)
# set_xla(conf.xla)

Here we get a list of data and label filenames from local storage.

In [ ]:
# Get data and label filenames for training
data_filenames = get_dataset_filenames(images_dir, ext='*.npy')
label_filenames = get_dataset_filenames(labels_dir, ext='*.npy')
assert len(data_filenames) == len(label_filenames), \
    'Number of data and label filenames do not match'
print(f'Data: {len(data_filenames)}, Label: {len(label_filenames)}')

## Small Exploratory Data Analysis

In [ ]:
test_image = np.load(data_filenames[-100])
test_label = np.load(label_filenames[-100])

In [ ]:
np.unique(test_label)

Lets look at the occurrence of labels in the dataset. Only do this when you have enough RAM to load the dataset into memory. Change this method to areggation if you have more tiles than the total RAM.

In [ ]:
val_labels = []
for label_filename in label_filenames:

    # read label
    val_labels.append(np.squeeze(np.load(label_filename).astype(np.uint8)))

val_labels = np.array(val_labels)
val_labels.shape

Below is the number of occurrences per class.

In [ ]:
np.unique(val_labels, return_counts=True)

NOTE: There seems to be no-data on the dataset. Remove it before proceeding with training. This will certainly give you a NaN loss.

Once the tiles are ready, we create the dataset lists again

In [ ]:
# Setup dataloader, modify data loader for this new dataset
class RegressionDataLoaderSRLite(RegressionDataLoader):
    
    # we modify the load_data function for this use case
    def load_data(self, x, y):
        """
        Load data on training loop.
        """

        # Read data
        x = np.moveaxis(np.load(x), 0, -1)
        y = np.moveaxis(np.load(y), 0, -1)
        
        # Normalize labels, default is diving by 1.0
        x = normalize_image(x, self.conf.normalize)
        #y = normalize_image(y, self.conf.normalize_label)

        # Simple standardization, replace based on your own project
        if self.conf.standardization is not None:
            x = standardize_image(
                x, self.conf.standardization, self.mean, self.std)

        # Augment
        if self.conf.augment:

            if np.random.random_sample() > 0.5:
                x = np.fliplr(x)
                y = np.fliplr(y)
            if np.random.random_sample() > 0.5:
                x = np.flipud(x)
                y = np.flipud(y)
            if np.random.random_sample() > 0.5:
                x = np.rot90(x, 1)
                y = np.rot90(y, 1)
            if np.random.random_sample() > 0.5:
                x = np.rot90(x, 2)
                y = np.rot90(y, 2)
            if np.random.random_sample() > 0.5:
                x = np.rot90(x, 3)
                y = np.rot90(y, 3)

        return x, y

In [ ]:
# Set main data loader
main_data_loader = RegressionDataLoaderSRLite(
    data_filenames, label_filenames, conf
)

In [ ]:
# Get and compile the model
model = get_model(conf.model)
model.compile(
    loss=get_loss(conf.loss),
    optimizer=get_optimizer(conf.optimizer)(conf.learning_rate),
    metrics=get_metrics(conf.metrics)
)

model.summary()

In [ ]:
# Fit the model and start training
model.fit(
    main_data_loader.train_dataset,
    validation_data=main_data_loader.val_dataset,
    epochs=conf.max_epochs,
    steps_per_epoch=main_data_loader.train_steps,
    validation_steps=main_data_loader.val_steps,
    callbacks=get_callbacks(conf.callbacks)
)

## Inference

Simple inference of small tiles. You will need the full pipeline to run over large scenes. For the actual use case, add a new directory of tiles to validate against.

In [ ]:
validation_data_dir = '/explore/nobackup/people/jacaraba/projects/SRLite/datasets/regression-dataset'
val_images_dir = os.path.join(conf.data_dir, 'images')
val_labels_dir = os.path.join(conf.data_dir, 'labels')

Get the tiles to perform validation on:

In [ ]:
val_data_filenames = get_dataset_filenames(val_images_dir, ext='*.npy')
val_label_filenames = get_dataset_filenames(val_labels_dir, ext='*.npy')

Gather all tiles into a single array.

In [ ]:
val_data = []
val_labels = []
for data_filename, label_filename in zip(val_data_filenames, val_label_filenames):
    
    # read data
    val_data_tile = np.load(data_filename)
    if conf.standardization is not None:
        val_data_tile = standardize_image(
            val_data_tile, conf.standardization, conf.mean, conf.std)
    
    # read label
    val_data_label = np.load(label_filename)
    
    val_data.append(val_data_tile)
    val_labels.append(val_data_label)

val_data = np.array(val_data)
val_labels = np.array(val_labels)

In [ ]:
val_data = np.moveaxis(val_data, 1, -1)
val_labels = np.moveaxis(val_labels, 1, -1)
val_data.shape, val_labels.shape

In [ ]:
plt.hist(np.squeeze(val_labels.flatten()), bins='auto')
plt.show()

Load the tensorflow model and predict

In [ ]:
# Load model for inference
model = load_model(
    model_filename=conf.model_filename,
    model_dir=conf.model_dir
)

In [ ]:
predictions = model.predict(val_data, batch_size=128)
predictions.shape

In [ ]:
results = model.evaluate(val_data, val_labels, batch_size=128)
results

## Visualize Some Predictions

In [ ]:
predictions = np.squeeze(predictions)
val_labels = np.squeeze(val_labels)

In [ ]:
fig, axs = plt.subplots(4, 2, figsize=(15, 15))
images_list = [0, 1, 2, 3]
for ax_id, img_id in zip(range(len(axs)), images_list):
    axs[ax_id, 0].imshow(predictions[img_id, :, :])
    axs[ax_id, 1].imshow(val_labels[img_id, :, :])
plt.tight_layout()

See those edge effects? The model does not have enough spatial information on the boundaries, thus it predicts them erroneously. We normally clip the boundaries by a given stride when assessing accuracy in small tiles. We stitch them together when doing the prediction of larger scenes.